# GTF validator

## Richiesta
Si richiede di scrivere un validatore del formato GTF (Gene Transfer Format) che prenda in input un file in formato GTF che annota un set di geni ed effettui la validazione del file rispetto alla specifica del formato.<br>
Il validatore deve produrre in output un report con le violazioni presenti, specificando per ognuna di esse il record che la contiene (posizione all'interno del file in input) e tutte le informazioni che si ritengono necessarie per descriverla e correggerla.<br>
Il validatore può essere prodotto sia come script che come Jupyter Notebook, e deve essere adeguatamente commentato. Si richiede inoltre un documento che elenchi e descriva brevemente le violazioni che sono state considerate. Per ogni violazione considerata, includere un file con tale violazione.

## Input
Per eseguire questo notebook è necessario fornire il file GTF da testare collocandolo nella stessa directory del notebook. È possibile indicare il nome del file nella prima cella di codice (il nome di default è `input.gtf`). È naturalmente possibile indicare il percorso di un file se non si desidera spostarlo nella cartella di lavoro del notebook.

## Output
Dopo aver analizzato il file, il notebook crea un report delle violazioni (posizione e indicazioni aggiuntive) che viene stampato a schermo alla fine del notebook.

***

In [ ]:
# Modificare la seguente riga per specificare il file da analizzare:
input_file_name = './input.gtf'

Durante la lettura del file tutte le violazioni vengono inserite nella lista `violations` che verrà poi usata per stampare un report alla fine dell'esecuzione. Ogni elemento della lista è a sua volta una lista che contiene la riga affetta e una breve descrizione del problema.

In [ ]:
# Inizializzazione della lista di violazioni:
violations = []

L'istruzione seguente importa i moduli necessari.

In [ ]:
import re

Le feature "CDS", "start_codon" e "stop_codon" sono richieste in ogni file GTF, mentre le altre sono opzionali. Per verificare la loro presenza vengono usati tre flag booleani. Essi sono contenuti nel dizionario `required_features` e vengono settati a `True` se la relativa feature appare almeno una volta nel file.

In [ ]:
required_features = {"CDS": False, "start_codon": False, "stop_codon": False}

### Definizione delle funzioni per il controllo dei campi di ogni riga del file
Le otto funzioni seguenti servono a verificare la correttezza sintattica dei diversi campi che si possono trovare in una riga. Esse seguono tutte la stessa logica: ricevono in input uno o più campi di una riga e il numero della riga stessa. Dopo aver effettuato i controlli, per esempio cercando un match con una espressione regolare, ogni funzione può segnalare una violazione effettuando un append alla lista `violations`, inserendo quindi il numero della riga affetta (ricevuto come parametro) e una breve descrizione dell'anomalia.<br>
Il funzionamento specifico e le scelte implementative che hanno guidato la scrittura delle seguenti funzioni sono descritte nel PDF di documentazione, dove si descrivono inoltre i vari tipi di violazioni considerate.

In [ ]:
def check_seqname_violation(row_number, seqname):
    if not re.match('^[^\s]+$', seqname):
        violations.append([row_number, 'Il campo "seqname" contiene caratteri non ammessi'])

In [ ]:
def check_source_violation(row_number, source):
    if not re.match('^[^\s]+$', source):
        violations.append([row_number, 'Il campo "source" contiene caratteri non ammessi'])

In [ ]:
def check_feature_violation(row_number, feature):
    # Per prima cosa viene controllata la presenza delle tre feature obbligatorie,
    # eventualmente settando il flag opportuno.
    if feature in ["CDS", "start_codon", "stop_codon"]:
        required_features[feature] = True
        return

    if feature not in ["5UTR", "3UTR", "inter", "inter_CNS", "intron_CNS", "exon"]:
        violations.append([row_number, '(warning) La feature "' + feature + '" non è riconosciuta'])

In [ ]:
def check_start_end_violation(row_number, start, end):
    not_integer_found = False
    # Controllo correttezza formale start e end:
    if not re.match('^\d+$', start):
        violations.append([row_number, 'Il campo "start" contiene caratteri non ammessi'])
        not_integer_found = True
    if not re.match('^\d+$', end):
        violations.append([row_number, 'Il campo "end" contiene caratteri non ammessi'])
        not_integer_found = True
    if not_integer_found:
        return

    # Conversione a int e verifica valori:
    start = int(start)
    end = int(end)
    if start < 1:
        violations.append([row_number, 'Il campo "start" assume un valore non consentito'])
    if end < 1:
        violations.append([row_number, 'Il campo "end" assume un valore non consentito'])
    if start > end:
        violations.append([row_number, 'Il campo "start" è maggiore del campo "end"'])

In [ ]:
def check_score_violation(row_number, score):
    if not re.match('(?:^-?\d+$)|(?:^-?\d+\.\d+$)|(?:^\.$)', score):
        violations.append([row_number, 'Il campo "score" contiene un valore non ammesso'])

In [ ]:
def check_strand_violation(row_number, strand):
    if strand != '+' and strand != '-':
        violations.append([row_number, 'Il campo "strand" contiene un valore non ammesso'])

In [ ]:
def check_frame_violation(row_number, feature, frame):
    if feature in ["CDS", "start_codon", "stop_codon"]:
        if frame not in ["0", "1", "2"]:
            violations.append([row_number, 'Il campo "frame" contiene un valore non ammesso per questa feature'])
    else:
        if frame != ".":
            violations.append([row_number, 'Il campo "frame" contiene un valore non ammesso per questa feature'])

In [ ]:
def check_attributes_violation(row_number, feature, attributes):
    if feature == "inter" or feature == "inter_CNS":
        # In questo caso l'attributo 'transcript_id' deve essere vuoto (ma comunque presente):
        if not re.match('(?:^gene_id "[^"]*"; transcript_id "";(?: \w+ "[^"]*";)*)|(?:^transcript_id ""; gene_id "[^"]*";(?: \w+ "[^"]*";)*)', attributes):
            violations.append([row_number, 'Il campo "attributes" non è valido per questa feature'])
    else:
        # In questo caso 'transcript_id' deve avere un valore associato:
        if not re.match('(?:^gene_id "[^"]*"; transcript_id "[^"]+";(?: \w+ "[^"]*";)*)|(?:^transcript_id "[^"]+"; gene_id "[^"]*";(?: \w+ "[^"]*";)*)', attributes):
            violations.append([row_number, 'Il campo "attributes" non è valido per questa feature'])

### Definizione di funzioni varie per il controllo del file nella sua interezza

La funzione `validate_row` effettua il controllo di una singola riga. Oltre alla riga stessa, riceve come parametro il suo numero (posizione all'interno del file). Tale parametro è necessario nel caso in cui la riga contenga una violazione, infatti il numero della riga deve essere inserito nella lista di violazioni.<br>
Questa funzione verrà chiamata per ogni riga presente nel file e si appoggia alle otto funzioni descritte in precedenza per la verifica dei diversi campi.

In [ ]:
def validate_row(row, row_number):
    # All'interno dei file GTF sono ammessi commenti, identificati dal carattere
    # cancelletto iniziale. Un commento può occupare una riga intera (in questo
    # caso il cancelletto è il primo carattere della riga) o può trovarsi alla fine
    # di una riga. In ogni caso, tutto ciò che si trova dopo un carattere cancelletto
    # non deve essere parsato, quindi dalla riga vengono rimossi tutti i caratteri che
    # seguono un cancelletto.
    row = row.split('#', 1)[0]
    # Effettuiamo inoltre uno strip per rimuovere i caratteri indesiderati al
    # termine della riga:
    row = row.rstrip()
    # Possiamo scartare le righe che dopo le operazioni precedenti sono rimaste
    # vuote, come ad esempio righe che erano adibite a commenti o che erano
    # intenzionalmente senza contenuto:
    if row == '':
        return

    # Controllo sul numero minimo di campi presenti in una riga:
    fields = row.split('\t')
    if len(fields) != 9:
        violations.append([row_number, "Ogni record deve contenere nove campi, ne sono stati trovati solo " + str(len(fields))])
        return

    # Verifica della correttezza dei singoli campi della riga:
    check_seqname_violation(row_number, fields[0])
    check_source_violation(row_number, fields[1])
    check_feature_violation(row_number, fields[2])
    check_start_end_violation(row_number, fields[3], fields[4])
    check_score_violation(row_number, fields[5])
    check_strand_violation(row_number, fields[6])
    check_frame_violation(row_number, fields[2], fields[7])
    check_attributes_violation(row_number, fields[2], fields[8])

La funzione `validate_required_features` verrà chiamata dopo che sono state parsate tutte le righe del file. Essa verifica tramite la lettura del dizionario `required_features` se le tre feature obbligatorie (CDS, start_codon e stop_codon) sono presenti almeno una volta all'interno del file, come richiesto dal formato GTF.<br>
Nel caso in cui ci siano feature obbligatorie mancanti, viene segnalato un errore alla riga "speciale" zero.

In [ ]:
def validate_required_features():
    for f in ["CDS", "start_codon", "stop_codon"]:
        if not required_features[f]:
            violations.append([0, 'La feature obbligatoria "' + f + '" non è presente nel file'])

La funzione `print_results` stampa il report di violazioni leggendo la lista `violations`. Essa è l'ultima funzione ad essere chiamata.

In [ ]:
def print_results():
    if len(violations) == 0:
        print("Non sono presenti violazioni nel file GTF di input.")
    else:
        print("Sono state individuate le seguenti violazioni:")
        for v in violations:
            print("Riga " + str(v[0]) + ": " + v[1])

### Apertura del file e inizio dell'analisi

L'impossibilità di aprire un file costituisce una "violazione", seppur banale. Se non viene trovato nessun file con il nome specificato l'eccezione viene gestita inserendo nella lista delle violazioni una entry corrispondente alla riga `0` (in questo caso `0` è usato come valore speciale per identificare una violazione che non riguarda nessuna riga in particolare).

In modo del tutto analogo, se il file viene aperto ma risulta vuoto, viene aggiunta alla lista una "violazione" banale simile alla precedente (sempre alla riga `0`). Prima di effettuare il controllo, viene invocata la funzione `strip()` su tutte le righe e quelle vuote vengono escluse. In questo modo, un file che contiene solamente ritorni a capo e spazi viene di fatto considerato vuoto.

Se queste violazioni banali non vengono trovate, si procede all'analisi delle righe del file tramite la funzione `validate_row`.

In [ ]:
# Apertura del file di input:
try:
    with open(input_file_name, 'r') as input_file:
        file_rows = input_file.readlines()
    # Viene controllato se il file è vuoto:
    if len([row for row in file_rows if row.strip() != '']) == 0:
        violations.append([0, "Il file risulta vuoto"])
    else:
        # Viene controllata la validità di ogni riga:
        for r in range(len(file_rows)):
            validate_row(file_rows[r], r+1)
        # Viene controllata la presenza delle feature richieste:
        validate_required_features()
except:
    violations.append([0, "File non trovato"])

### Output report conclusivo
Al termine dell'analisi del file la lista delle violazioni contiene le informazioni necessarie alla generazione del report finale. Viene chiamata la funzione `print_results` per leggere la lista e stampare a schermo il report delle violazioni.

In [ ]:
# Print del report alla fine dell'analisi:
print_results()